# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [11]:
#include <stdio.h>

#define S 200.0
#define N 5
#define J 1.0 //Em porcentagem

int main(){
    float parcela = S;
    for(int i = 0; i < N; i++){
        printf("%.2f\n", parcela);
        parcela *= (1 + (J/100));
    }
        
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 2 - Escrevendo um Módulo

Reescreva o código acima de forma que seu programa faça uso de uma função que seja responsável pelo cálculo de uma parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal.

In [14]:
#include <stdio.h>

#define S 200.0
#define N 5
#define J 1.0 //Em porcentagem

float parcelaX(float inicial, int parcela, float juros){
    float resultado = inicial;
    float taxa = 1 + juros/100;
    for(int i = 0; i < parcela; i++) 
        resultado *= taxa;
    
    return resultado;
}

int main(){
    for(int i = 0; i < N; i++)
        printf("%.2f\n", parcelaX(S, i, J));
        
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 3 - Minimizando os Parâmetros

Modifique o módulo para que o cliente (que chama a função) use o mínimo de parâmetros possíveis. Para isso a sua função precisa estar ciente do estado corrente do empréstimo, ou seja, ela precisará saber qual a próxima parcela que ela precisa calcular sem que lhe seja informado, para isso:
* evite informar recorrentemente como parâmetros dados sobre as características do empréstimo;
* evite usar informações de controle (e.g., parcela corrente) como parâmetro.

In [1]:
#include <stdio.h>

#define S 200.0
#define N 5
#define J 1.0 //Em porcentagem

float valor_atual = 0;

float parcelaX(float inicial, float taxa){
   if(valor_atual == 0)
        valor_atual = inicial;
    else
        valor_atual *= taxa;
    
    return valor_atual;
}

int main(){
    float taxa = 1 + J/100;
    for(int i = 0; i < N; i++)
        printf("%.2f\n", parcelaX(S, taxa));
        
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Partindo da Solução da Parte 2, modifique a forma como você representa o empréstimo, de forma que a função que calcula a próxima parcela receba um único parâmetro.

In [2]:
#include <stdio.h>

#define S 200.0
#define N 5
#define J 1.0 //Em porcentagem

float parcelaX(float* dados){
    float resultado = dados[0];
    float taxa = 1 + dados[2]/100;
    for(int i = 0; i < dados[1]; i++) 
        resultado *= taxa;
    
    return resultado;
}

int main(){
    float emprestimo[3];
    emprestimo[0] = S;
    emprestimo[1] = 0;
    emprestimo[2] = J;
    
    for(int i = 0; i < N; i++){
        printf("%.2f\n", parcelaX(emprestimo));
        emprestimo[1] = i + 1;
    }
        
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. O seu programa deve ser capaz de informar as parcelas de mais de um empréstimo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar o programa principal, a função ou ambos para atender esse requisito da melhor forma possível.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [9]:
#include <stdio.h>
#include <stdlib.h>

#define MAX_NUM_EMPRESTIMOS 10

int indice_emprestimo_atual = 0;

typedef struct emprestimo{
    float valor_atual;
    int num_parcelas;
    float juros;
    
    int parcelaAtual; //valor padrão inicial = 0
} Emprestimo;

Emprestimo* criarEmprestimo(float valor_inicial, int num_parcelas, float juros, Emprestimo** emprestimos){
    if(indice_emprestimo_atual == MAX_NUM_EMPRESTIMOS) return NULL;
    Emprestimo* e = malloc(sizeof(Emprestimo));
    
    e->valor_atual = valor_inicial;
    e->num_parcelas = num_parcelas;
    e->juros = juros;
    
    emprestimos[indice_emprestimo_atual] = e;
    indice_emprestimo_atual += 1;
    
    return e;
}

/** Retorna 1 se há mais parcelas a serem calculadas, e retorna 0 caso contrário
*/
int calcularProxParcela(Emprestimo* emprestimo, int num_emprestimo){
    if(emprestimo == NULL || emprestimo->parcelaAtual == emprestimo->num_parcelas) return 0;
    
    emprestimo->parcelaAtual += 1;
    
    printf("Empréstimo %d: parcela %d eh %.02f\n", num_emprestimo, emprestimo->parcelaAtual, emprestimo->valor_atual);
    
    emprestimo->valor_atual *= 1 + emprestimo->juros / 100;
    
    return 1;
}

int main(){
    Emprestimo** emprestimos = malloc(MAX_NUM_EMPRESTIMOS*sizeof(Emprestimo*));
    
    Emprestimo* e1 = criarEmprestimo(200.0, 5, 1.0, emprestimos);
    Emprestimo* e2 = criarEmprestimo(500.0, 7, 2.0, emprestimos);
    Emprestimo* e3 = criarEmprestimo(600.0, 2, 5.0, emprestimos);
    
    float temp;
    do{
        temp = 0;
        for(int i = 0; i < MAX_NUM_EMPRESTIMOS; i++){
            temp += calcularProxParcela(emprestimos[i], i + 1);
        }
        
    }while(temp != 0);
    
    for(int i = 0; i < indice_emprestimo_atual; i++){
        free(emprestimos[i]);
    }
    
    free(emprestimos);
        
    return 0;
}

Empréstimo 1: parcela 1 eh 200.00
Empréstimo 2: parcela 1 eh 500.00
Empréstimo 3: parcela 1 eh 600.00
Empréstimo 1: parcela 2 eh 202.00
Empréstimo 2: parcela 2 eh 510.00
Empréstimo 3: parcela 2 eh 630.00
Empréstimo 1: parcela 3 eh 204.02
Empréstimo 2: parcela 3 eh 520.20
Empréstimo 1: parcela 4 eh 206.06
Empréstimo 2: parcela 4 eh 530.60
Empréstimo 1: parcela 5 eh 208.12
Empréstimo 2: parcela 5 eh 541.22
Empréstimo 2: parcela 6 eh 552.04
Empréstimo 2: parcela 7 eh 563.08
